<a href="https://colab.research.google.com/github/Gauthampharish/OCR/blob/main/Copy_of_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install keras-tuner

In [10]:

!pip install emnist

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import emnist


In [12]:

from emnist import extract_training_samples
train_images, train_labels = extract_training_samples('byclass')

In [13]:

from emnist import extract_test_samples
test_images, test_labels = extract_test_samples('byclass')


In [14]:

train_images = tf.keras.utils.normalize(train_images, axis = 1)
test_images = tf.keras.utils.normalize(test_images, axis = 1)

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

In [15]:

rotation_range_val = 15
width_shift_val = 0.10
height_shift_val = 0.10

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range = rotation_range_val,
                             width_shift_range = width_shift_val,
                             height_shift_range = height_shift_val)

In [17]:
train_datagen.fit(train_images.reshape(train_images.shape[0], 28, 28, 1))

In [18]:
test_datagen = ImageDataGenerator()
test_datagen.fit(test_images.reshape(test_images.shape[0], 28, 28, 1))
test_datagen

In [19]:
# Define the model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Reshape((-1, 64)))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(62, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [20]:

# Import necessary libraries
from kerastuner.tuners import RandomSearch
import tensorflow as tf

# Set up the tuner with a temporary directory in Colab
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='/content/drive/MyDrive/colab_temp',  # You can change this directory as needed
    project_name='helloworld'
)


In [21]:
from keras.utils import to_categorical

train_labels_one_hot = to_categorical(train_labels, num_classes=62)
test_labels_one_hot = to_categorical(test_labels, num_classes=62)

In [ ]:
# Search for the best hyperparameters
tuner.search(train_datagen.flow(train_images, train_labels_one_hot, batch_size=1024),
             validation_data=test_datagen.flow(test_images, test_labels_one_hot, batch_size=32),
             epochs=8)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
416               |416               |units
0.001             |0.001             |learning_rate

Epoch 1/8
682/682 [==============================] - 979s 1s/step - loss: 1.4353 - accuracy: 0.5911 - val_loss: 0.5667 - val_accuracy: 0.8045
Epoch 2/8
682/682 [==============================] - 965s 1s/step - loss: 0.6410 - accuracy: 0.7857 - val_loss: 0.4595 - val_accuracy: 0.8350
Epoch 3/8
682/682 [==============================] - 980s 1s/step - loss: 0.5454 - accuracy: 0.8122 - val_loss: 0.4477 - val_accuracy: 0.8393
Epoch 4/8
682/682 [==============================] - 940s 1s/step - loss: 0.4993 - accuracy: 0.8256 - val_loss: 0.4110 - val_accuracy: 0.8499
Epoch 5/8
682/682 [==============================] - 965s 1s/step - loss: 0.4746 - accuracy: 0.8325 - val_loss: 0.3939 - val_accuracy: 0.8541
Epoch 6/8
 78/682 [==>...........................] - ETA: 13:08 - loss: 0.4609 - accuracy: 0.8361

KeyboardInterrupt: ignored

In [22]:
from tensorflow.keras.models import load_model

# Replace 'saved_model_path' with the path to your saved model directory
saved_model_path = '/content/your_model.h5'

# Load the model
model = load_model(saved_model_path)

In [23]:
# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

NameError: ignored

In [24]:
# Train the model with the training data
model.fit(train_datagen.flow(train_images, train_labels_one_hot, batch_size=1024), epochs=15)


Epoch 1/15
682/682 [==============================] - 196s 265ms/step - loss: 1.5528 - accuracy: 0.5720
Epoch 2/15
682/682 [==============================] - 175s 257ms/step - loss: 0.7229 - accuracy: 0.7679
Epoch 3/15
682/682 [==============================] - 179s 262ms/step - loss: 0.6070 - accuracy: 0.7996
Epoch 4/15
682/682 [==============================] - 178s 261ms/step - loss: 0.5514 - accuracy: 0.8150
Epoch 5/15
682/682 [==============================] - 179s 262ms/step - loss: 0.5200 - accuracy: 0.8236
Epoch 6/15
682/682 [==============================] - 180s 265ms/step - loss: 0.4985 - accuracy: 0.8294
Epoch 7/15
682/682 [==============================] - 178s 261ms/step - loss: 0.4826 - accuracy: 0.8337
Epoch 8/15
682/682 [==============================] - 181s 265ms/step - loss: 0.4710 - accuracy: 0.8369
Epoch 9/15
682/682 [==============================] - 178s 261ms/step - loss: 0.4589 - accuracy: 0.8399
Epoch 10/15
682/682 [==============================] - 180s 264m

In [25]:

# Evaluate the model with the testing data
model.evaluate(test_datagen.flow(test_images, test_labels_one_hot, batch_size=32))

3636/3636 [==============================] - 17s 5ms/step - loss: 0.3700 - accuracy: 0.8624


[0.36999258399009705, 0.8623573780059814]

In [26]:
from tensorflow.keras.models import load_model, save_model

# Save the model
model.save("your_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('/content/new.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

characters = []
for contour in contours:
    # Get bounding box for each contour
    x, y, w, h = cv2.boundingRect(contour)

    # Extract the character
    character = gray[y:y+h, x:x+w]

    # Resize to a standard size (e.g., 28x28)
    character = cv2.resize(character, (28, 28))

    characters.append(character)

print(f'Number of characters: {len(characters)}')

# Save the characters as numpy arrays
characters_array = []
for i, char in enumerate(characters):
    # Convert the character to a numpy array
    char_array = np.array(char)

    # Append the character array to the list of characters
    characters_array.append(char_array)

# Convert the list of characters to a numpy array
characters_array = np.array(characters_array)



Number of characters: 15


In [28]:
def index_to_char(index):
    if index < 10:
        return str(index)
    elif index < 36:
        return chr(index - 10 + ord('A'))
    else:
        return chr(index - 36 + ord('a'))

# Predict the character for each image
for i, char_img in enumerate(characters_array):
    # Preprocess the image
    char_img = char_img.reshape(1, 28, 28, 1)
    char_img = char_img.astype('float32')
    char_img /= 255

    # Predict the character
    prediction = model.predict([char_img])
    predicted_index = np.argmax(prediction)
    predicted_char = index_to_char(predicted_index)

    print(f'Character {i}: {predicted_char}')

1/1 [==============================] - 1s 511ms/step
Character 0: a
1/1 [==============================] - 0s 20ms/step
Character 1: a
1/1 [==============================] - 0s 21ms/step
Character 2: a
1/1 [==============================] - 0s 20ms/step
Character 3: S
1/1 [==============================] - 0s 19ms/step
Character 4: e
1/1 [==============================] - 0s 21ms/step
Character 5: a
1/1 [==============================] - 0s 20ms/step
Character 6: M
1/1 [==============================] - 0s 19ms/step
Character 7: e
1/1 [==============================] - 0s 18ms/step
Character 8: a
1/1 [==============================] - 0s 20ms/step
Character 9: a
1/1 [==============================] - 0s 20ms/step
Character 10: 2
1/1 [==============================] - 0s 19ms/step
Character 11: S
1/1 [==============================] - 0s 18ms/step
Character 12: S
1/1 [==============================] - 0s 22ms/step
Character 13: r
1/1 [==============================] - 0s 20ms/step
Chara